In [1]:
import gym

# Create the Cart-Pole environment
env = gym.make("CartPole-v1")

# Run episodes using a random policy
num_episodes = 5

for episode in range(num_episodes):
    state = env.reset()
    done = False
    total_reward = 0

    while not done:
        env.render()  # Render the environment (optional)
        action = env.action_space.sample()  # Random action
        next_state, reward, done, _, _ = env.step(action)  # Take action
        total_reward += reward

    print(f"Episode {episode + 1}: Total Reward = {total_reward}")

env.close()


Episode 1: Total Reward = 11.0
Episode 2: Total Reward = 16.0
Episode 3: Total Reward = 32.0
Episode 4: Total Reward = 29.0
Episode 5: Total Reward = 20.0


C:\Users\nskit\anaconda3\Lib\site-packages\gym\envs\classic_control\cartpole.py:211: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("CartPole-v1", render_mode="rgb_array")
  gym.logger.warn(
C:\Users\nskit\anaconda3\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [2]:
import numpy as np
import gym
import random

# Initialize environment
env = gym.make("CartPole-v1")
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

# Discretization of continuous state space (for Q-learning)
num_bins = 20  # Discretization levels per state variable
state_bins = [
    np.linspace(-4.8, 4.8, num_bins),  # Cart position
    np.linspace(-5, 5, num_bins),      # Cart velocity
    np.linspace(-0.418, 0.418, num_bins),  # Pole angle
    np.linspace(-5, 5, num_bins)       # Pole velocity
]

# Initialize Q-table
q_table = np.zeros((num_bins, num_bins, num_bins, num_bins, action_size))

# Hyperparameters
learning_rate = 0.1
discount_factor = 0.99
epsilon = 1.0  # Initial exploration rate
epsilon_decay = 0.995
min_epsilon = 0.01
num_episodes = 5000

# Function to discretize state
def discretize_state(state):
    discretized = [np.digitize(state[i], state_bins[i]) - 1 for i in range(len(state))]
    return tuple(discretized)

# Training Loop
for episode in range(num_episodes):
    state = discretize_state(env.reset()[0])
    done = False
    total_reward = 0

    while not done:
        # Choose action using epsilon-greedy policy
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()  # Explore
        else:
            action = np.argmax(q_table[state])  # Exploit
        
        # Take action and observe reward
        next_state, reward, done, _, _ = env.step(action)
        next_state = discretize_state(next_state)
        
        # Update Q-value using the Q-learning update rule
        best_next_action = np.argmax(q_table[next_state])
        q_table[state + (action,)] += learning_rate * (reward + discount_factor * q_table[next_state + (best_next_action,)] - q_table[state + (action,)])

        state = next_state
        total_reward += reward

    # Reduce epsilon (exploration) over time
    epsilon = max(min_epsilon, epsilon * epsilon_decay)

    if (episode + 1) % 500 == 0:
        print(f"Episode {episode + 1}, Total Reward: {total_reward}, Epsilon: {epsilon:.3f}")

env.close()


Episode 500, Total Reward: 36.0, Epsilon: 0.082
Episode 1000, Total Reward: 30.0, Epsilon: 0.010
Episode 1500, Total Reward: 40.0, Epsilon: 0.010
Episode 2000, Total Reward: 37.0, Epsilon: 0.010
Episode 2500, Total Reward: 37.0, Epsilon: 0.010
Episode 3000, Total Reward: 38.0, Epsilon: 0.010
Episode 3500, Total Reward: 45.0, Epsilon: 0.010
Episode 4000, Total Reward: 31.0, Epsilon: 0.010
Episode 4500, Total Reward: 37.0, Epsilon: 0.010
Episode 5000, Total Reward: 41.0, Epsilon: 0.010
